# 5th attempt - RNN

In [62]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [63]:
SIZE = 10
AMOUNT_BOARDS = 1000

In [64]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [65]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [66]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 41366
reverse df sort: 41366


In [67]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  33506
len x test:  4137
len y train:  33506
len y test:  4137


In [68]:
X_train.shape

(33506, 100)

In [69]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [70]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 100)
(33506,)


In [71]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [72]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 10, 10, 1)
(33506, 1)


In [73]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [74]:
import tensorflow as tf

# הנחה: הנתונים שלך כבר בטעינה בשם:
# X_train_array: (33506, 10, 10, 1)
# y_train_array: (33506, 1)

INPUT_DIM = SIZE * SIZE  
TIMESTEPS = gen-1

# שינוי צורת הנתונים כך שתתאים ל-LSTM
X_train = X_train_array.reshape((-1, TIMESTEPS, INPUT_DIM)).astype('float32')
y_train = y_train_array.astype('float32')

print("Reshaped X_train:", X_train.shape)
print("y_train:", y_train.shape)

# הגדרת המודל
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(TIMESTEPS, INPUT_DIM)),
    tf.keras.layers.LSTM(128, activation='tanh'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # סיווג בינארי
])


Reshaped X_train: (33506, 1, 100)
y_train: (33506, 1)


In [75]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 125,569 (490.50 KB)

 Trainable params: 125,569 (490.50 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
# אימון
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7977 - loss: 0.4632 - val_accuracy: 0.8078 - val_loss: 0.3977
Epoch 2/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8200 - loss: 0.3832 - val_accuracy: 0.8154 - val_loss: 0.3862
Epoch 3/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8289 - loss: 0.3606 - val_accuracy: 0.8198 - val_loss: 0.3748
Epoch 4/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8457 - loss: 0.3342 - val_accuracy: 0.8248 - val_loss: 0.3698
Epoch 5/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8576 - loss: 0.3189 - val_accuracy: 0.8284 - val_loss: 0.3712
Epoch 6/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8674 - loss: 0.2973 - val_accuracy: 0.8295 - val_loss: 0.3708
Epoch 7/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8794 - loss: 0.2746 - val_accuracy: 0.8278 - val_loss: 0.3808
Epoch 8/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8932 - loss: 0.2482 - val_accuracy: 0.

In [77]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_3_1/Cast:0", shape=(None, 100), dtype=float32). Expected shape (None, 1, 100), but input has incompatible shape (None, 100)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 100), dtype=int32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
def evaluate_model(model, X_test_array, y_test_array):

    # predict test
    y_pred = model.predict(X_test_array)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Confusion matrix
    cm = confusion_matrix(y_test_array, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    # calc the parameters
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    acc = (tp + tn) / (tp + tn + fp + fn)

    # print in table
    print("\n===== Evaluation Results =====")
    print("┌──────────────┬────────────┬────────────┐")
    print("│              │ Pred=Alive │ Pred=Dead  │")
    print("├──────────────┼────────────┼────────────┤")
    print(f"│ True=Alive   │ {tp:10d} │ {fn:10d} │")
    print(f"│ True=Dead    │ {fp:10d} │ {tn:10d} │")
    print("└──────────────┴────────────┴────────────┘")

    print("\n--- Performance Metrics ---")
    print(f"{'Accuracy':<12}: {acc:.3f}")
    print(f"{'Precision':<12}: {precision:.3f}")
    print(f"{'Recall':<12}: {recall:.3f}")
    print(f"{'F1-score':<12}: {f1:.3f}")

In [ ]:
evaluate_model(model, X_test_array, y_test_array)

ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 10 and 100 for '{{node sequential_1_1/lstm_1_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1_1/lstm_1_1/strided_slice_1, sequential_1_1/lstm_1_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [32,10], [100,512].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(32, 10), dtype=float32)
  • states=('tf.Tensor(shape=(32, 128), dtype=float32)', 'tf.Tensor(shape=(32, 128), dtype=float32)')
  • training=False